## POC client notebook

Get the current token from the Pod's filesystem

In [1]:
def get_current_token():
    try:
        with open(
            "/var/run/secrets/kubernetes.io/serviceaccount/token", "r"
        ) as file:
            token = file.read().strip()
        return token
    except Exception as e:
        print(f"Error reading token: {e}")
        return None


token = get_current_token()

Decode the token for your investigation

In [2]:
import jwt
jwt.decode(token, options={"verify_signature": False})

{'aud': ['https://kubernetes.default.svc'],
 'exp': 1748439300,
 'iat': 1716903300,
 'iss': 'https://kubernetes.default.svc',
 'kubernetes.io': {'namespace': 'feast',
  'pod': {'name': 'feast-notebook-0',
   'uid': '911eb512-ad45-4301-ab09-34659cacda6c'},
  'serviceaccount': {'name': 'feast-notebook',
   'uid': '190310c8-293c-4468-87c5-e6f6f0e1b303'},
  'warnafter': 1716906907},
 'nbf': 1716903300,
 'sub': 'system:serviceaccount:feast:feast-notebook'}

A `Session` implementyation to always include the authorization token in the header

In [4]:
import requests
class AuthorizedSession(requests.Session):
    def __init__(self, token=None):
        super().__init__()
        self.token = token

    def request(self, method, url, *args, **kwargs):
        if self.token:
            kwargs.setdefault('headers', {})['Authorization'] = f'Bearer {self.token}'
        return super().request(method, url, *args, **kwargs)

A method to run a request on the defined `url`, starting from the deployed `server_url`.

Both GET and POST are attempted and results displayed.

In [9]:
server_url = "http://poc-app.feast.svc.cluster.local:8000"
def run_request(
                    session: AuthorizedSession,
                    url: str = f"{server_url}/a",
):
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"  # Adjust content type as needed
    }

    response = session.get(url, headers=headers)
    if response.status_code == 200:
        print("GET Response:", response.json())
    else:
        print("GET Error:", response.status_code, response.text)

    response = session.post(url, headers=headers)
    if response.status_code == 200:
        print("POST Response:", response.json())
    else:
        print("POST Error:", response.status_code, response.text)


Test endpoint `/a`

In [29]:
session = AuthorizedSession(get_current_token())
run_request(session, url=f"{server_url}/a")

GET Error: 403 {"message":"No permissions to execute [<AuthzedAction.READ: 'read'>] on AuthzedResourceType.A:a. Requires roles ['a-reader']"}
POST Response: {'message': 'edit_A'}


Test endpoint `/b`

In [26]:
run_request(session, url=f"{server_url}/b")

GET Error: 403 {"message":"No permissions to execute [<AuthzedAction.READ: 'read'>] on AuthzedResourceType.B:b. Requires roles ['b-reader', 'b-editor']"}
POST Error: 403 {"message":"No permissions to execute [<AuthzedAction.EDIT: 'edit'>] on AuthzedResourceType.B:b. Requires roles ['b-reader', 'b-editor']"}


Test unprotected endpoint `/`

In [27]:
run_request(session, url=f"{server_url}/")

GET Response: {'message': 'read_unprotected'}
POST Response: {'message': 'post_unprotected'}


Repeat the above tests by creating different role bindings, using:

In [28]:
!oc create rolebinding poc-a-editor --role=a-editor --serviceaccount=default:feast-notebook

rolebinding.rbac.authorization.k8s.io/poc-a-editor created
